In [30]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.linear_model import LinearRegression

In [2]:
df_cl_1 = pd.read_csv(r'/mnt/d/BusinessSuiteData/Mercari/clean_data/df_cl_1_22-09-30.csv')

In [3]:
random_seeds = [8,13,21]

Feature Selection

In [4]:
features = \
    ['item_condition_id',
     #'brand_name', 
     'price',
     'shipping',
     'category_name_0']

Set up

In [5]:
df_cl_1[features].head()

,item_condition_id,price,shipping,category_name_0
0,3,10.0,1,Men
1,3,52.0,0,Electronics
2,1,10.0,1,Women
3,1,35.0,1,Home
4,1,44.0,0,Women


X, Y, Train/Test

In [6]:
X = df_cl_1[features].drop(['price'], axis = 1)
y = df_cl_1['price'].copy()

In [7]:
X.dtypes

item_condition_id     int64
shipping              int64
category_name_0      object
dtype: object

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seeds[0])

Pipeline

In [9]:
column_transformer = ColumnTransformer([('one_hot_encoder', 
                                         OneHotEncoder(drop='first', 
                                                       sparse=False, 
                                                       handle_unknown='ignore'),
                                         make_column_selector(dtype_include=object)
                                        )], 
                                       remainder='passthrough')

X_train

,item_condition_id,shipping,category_name_0
105549,1,1,Beauty
1211475,3,0,Women
1299985,1,0,Home
1239728,2,1,Women
609080,3,1,Beauty
...,...,...,...
403592,3,0,Kids
324570,2,1,Beauty
231557,2,1,Kids
780649,2,0,Women


<center><h3>Modeling

In [21]:
scoring = ['r2', 'neg_mean_squared_error', 'neg_mean_squared_log_error']

Linear Regression

In [31]:
lr_pipeline = Pipeline([
    ('Column_transformer', column_transformer),
    # ('Scaler' , StandardScaler()),
    ('Linear_Regression', LinearRegression(fit_intercept=True, normalize='deprecated',
                                          copy_X=True, n_jobs=None, positive=False))
])

In [32]:
# Grid searches dummy_pipeline

lr_pipeline_params = [dict()]

    
lr_pipeline_gs = GridSearchCV(
                               lr_pipeline, 
                               lr_pipeline_params, 
                               cv=5, 
                               scoring=scoring,
                               return_train_score=False,
                               verbose=5,
                               n_jobs = -1,
                               refit = 'neg_mean_squared_log_error')

lr_pipeline_gs.fit(X_train, y_train);

Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [33]:
lr_pipeline_gs.best_score_

-0.6233011323041775

In [34]:
lr_pipeline_gs.cv_results_['mean_test_neg_mean_squared_error']
lr_pipeline_gs.cv_results_['mean_test_r2']

array([-1453.74755385])

array([0.02732525])

Ideas

In [14]:
# Price outliers => stratified sampling ?
# Brand name target encoding